In [12]:
import csv
import os

def get_user_data():
    # 사용자 데이터 입력 받기
    name = input("이름을 입력하세요: ")
    gender = input("성별을 입력하세요 (남성/여성): ").lower()
    age = int(input("나이를 입력하세요: "))
    height = float(input("키를 입력하세요 (cm): "))
    weight = float(input("체중을 입력하세요 (kg): "))
    waist_circumference_inch = float(input("허리 둘레를 입력하세요 (인치): "))  # 허리둘레 인치로 입력
    diabetes = input("당뇨가 있습니까? (예/아니오): ").lower() == '예'
    return name, gender, age, height, weight, waist_circumference_inch, diabetes

def calculate_bmr(gender, age, height, weight):
    # Mifflin-St Jeor Equation으로 BMR 계산
    if gender == '남성':
        bmr = 10 * weight + 6.25 * height - 5 * age + 5
    elif gender == '여성':
        bmr = 10 * weight + 6.25 * height - 5 * age - 161
    else:
        raise ValueError("성별은 '남성' 또는 '여성'으로 입력해야 합니다.")
    
    return bmr

def calculate_tdee(bmr, activity_level):
    # 활동 수준에 따른 TDEE 계산
    activity_multiplier = {
        '운동 안함': 1.2,
        '가벼운 운동': 1.375,
        '중간 운동': 1.55,
        '강한 운동': 1.725,
        '매우 강한 운동': 1.9
    }
    
    if activity_level not in activity_multiplier:
        raise ValueError("활동 수준은 '운동 안함', '가벼운 운동', '중간 운동', '강한 운동', '매우 강한 운동' 중 하나여야 합니다.")
    
    tdee = bmr * activity_multiplier[activity_level]
    return tdee

def calculate_bmi(weight, height):
    # BMI 계산 (체질량지수)
    height_m = height / 100  # cm를 m로 변환
    bmi = weight / (height_m ** 2)
    return bmi

def calculate_body_fat_percentage(gender, waist_circumference, height, weight):
    # 허리둘레와 체중을 기반으로 체지방률 추정 (일반적인 공식)
    waist_circumference_cm = waist_circumference * 2.54  # 인치를 cm로 변환
    if gender == '남성':
        body_fat = (waist_circumference_cm * 0.74) - (weight * 0.082) - 34.89
    elif gender == '여성':
        body_fat = (waist_circumference_cm * 0.74) - (weight * 0.082) - 44.74
    else:
        raise ValueError("성별은 '남성' 또는 '여성'으로 입력해야 합니다.")
    
    return body_fat

def calculate_nutrients(tdee, weight, diabetes, bmi, body_fat):
    # 영양소 섭취량 계산
    # 단백질: 체중의 1.6배 (g)
    protein = weight * 1.6  # 기본값 1.6g/kg
    # 지방: TDEE의 25% (단위: g)
    fat = tdee * 0.25 / 9  # 기본값 25% 사용
    # 탄수화물: 나머지 TDEE로 계산 (단위: g)
    carbs = (tdee - (protein * 4 + fat * 9)) / 4
    
    # 당뇨가 있을 경우 당분 제한
    if diabetes:
        sugar = min(carbs * 0.1, 25)  # 당분 제한, 일일 25g 이하로 설정
    else:
        sugar = carbs * 0.2  # 일반적인 경우에는 20%를 당분으로 설정
    
    # 식이섬유 권장량 (g)
    fiber = 25  # 일반적인 식이섬유 권장량 (성인 기준)
    
    # 나트륨 권장량 (mg)
    sodium = 2300  # 하루 나트륨 권장량 (mg)
    
    # 건강 지표에 따른 제한 적용
    if bmi >= 25:  # 비만인 경우
        protein *= 1.2  # 단백질을 약간 증가시켜서 근육량 유지
        carbs *= 0.8  # 탄수화물을 줄임
        sodium *= 0.8  # 나트륨 섭취를 줄임
    if body_fat > 30:  # 체지방률이 높을 경우
        protein *= 1.2  # 단백질을 더 섭취하여 근육량을 늘림
        carbs *= 0.7  # 탄수화물 섭취 제한
    
    return protein, fat, carbs, sugar, fiber, sodium

def save_to_csv(data):
    # CSV 파일로 데이터 저장
    file_exists = os.path.exists('user.csv')
    rows = []
    updated = False
    
    # 기존 데이터 읽기
    if file_exists:
        with open('user.csv', mode='r', newline='', encoding='utf-8') as file:
            reader = csv.reader(file)
            rows = list(reader)
    
    # 사용자 이름이 이미 존재하는지 확인
    for i, row in enumerate(rows):
        if row[0] == data[0]:  # 이름이 같은 경우
            rows[i] = data  # 해당 행을 업데이트
            updated = True
            break
    
    # 이름이 없다면 새로운 데이터를 추가
    if not updated:
        rows.append(data)
    
    # 파일에 데이터 저장
    with open('user.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(['이름', '성별', '나이', '키(cm)', '체중(kg)', '허리둘레(인치)', 'BMR(kcal)', 'TDEE(kcal)', 'BMI', '체지방량(kg)', '단백질(g)', '지방(g)', '탄수화물(g)', '당분(g)', '식이섬유(g)', '나트륨(mg)'])
        writer.writerows(rows)

def main():
    # 사용자 데이터를 입력받기
    name, gender, age, height, weight, waist_circumference, diabetes = get_user_data()
    
    # BMR 계산
    bmr = calculate_bmr(gender, age, height, weight)
    print(f"{name}님의 BMR(기초대사율)은 {bmr:.2f} kcal입니다.")
    
    # 활동 수준 입력받기
    activity_level = input("활동 수준을 입력하세요 ('운동 안함', '가벼운 운동', '중간 운동', '강한 운동', '매우 강한 운동'): ").lower()
    
    # TDEE 계산
    tdee = calculate_tdee(bmr, activity_level)
    print(f"{name}님의 TDEE(총 일일 에너지 소비량)는 {tdee:.2f} kcal입니다.")
    
    # BMI 계산
    bmi = calculate_bmi(weight, height)
    print(f"{name}님의 BMI(체질량지수)는 {bmi:.2f}입니다.")
    
    # 체지방률 계산
    body_fat = calculate_body_fat_percentage(gender, waist_circumference, height, weight)
    print(f"{name}님의 체지방량 추정치는 {body_fat:.2f}kg입니다.")
    
    # 영양소 섭취량 계산
    protein, fat, carbs, sugar, fiber, sodium = calculate_nutrients(tdee, weight, diabetes, bmi, body_fat)
    print(f"\n1일 권장 영양소 섭취량:")
    print(f"단백질: {protein:.2f} g")
    print(f"지방: {fat:.2f} g")
    print(f"탄수화물: {carbs:.2f} g")
    print(f"당분: {sugar:.2f} g")
    print(f"식이섬유: {fiber:.2f} g")
    print(f"나트륨: {sodium:.2f} mg")
    
    # 추가적인 정보 출력
    print("\n--- 추가적인 건강 지표 ---")
    if bmi < 18.5:
        print("BMI 상태: 저체중")
    elif 18.5 <= bmi < 24.9:
        print("BMI 상태: 정상 체중")
    elif 25 <= bmi < 29.9:
        print("BMI 상태: 과체중")
    else:
        print("BMI 상태: 비만")
    
    if body_fat < 10:
        print(f"체지방률 상태: 너무 낮음")
    elif body_fat >= 10 and body_fat < 20:
        print(f"체지방률 상태: 적정 체지방")
    elif body_fat >= 20 and body_fat < 30:
        print(f"체지방률 상태: 약간 높은 체지방")
    else:
        print(f"체지방률 상태: 높은 체지방")
    
    # CSV 파일로 저장
    data = [name, gender, age, height, weight, waist_circumference, bmr, tdee, bmi, body_fat, protein, fat, carbs, sugar, fiber, sodium]
    save_to_csv(data)
    print("사용자 데이터를 user.csv에 저장했습니다.")

if __name__ == "__main__":
    main()


조재투님의 BMR(기초대사율)은 1722.50 kcal입니다.
조재투님의 TDEE(총 일일 에너지 소비량)는 2067.00 kcal입니다.
조재투님의 BMI(체질량지수)는 27.68입니다.
조재투님의 체지방량 추정치는 26.22kg입니다.

1일 권장 영양소 섭취량:
단백질: 153.60 g
지방: 57.42 g
탄수화물: 207.65 g
당분: 25.00 g
식이섬유: 25.00 g
나트륨: 1840.00 mg

--- 추가적인 건강 지표 ---
BMI 상태: 과체중
체지방률 상태: 약간 높은 체지방
사용자 데이터를 user.csv에 저장했습니다.
